In [138]:
import matplotlib.pyplot as plt
from classiq import construct_combinatorial_optimization_model
from classiq.applications.combinatorial_optimization import OptimizerConfig, QAOAConfig
import numpy as np
from qiskit import QuantumCircuit as qc
from pyomo.environ import ConcreteModel, Var, Binary, minimize, Objective, ConstraintList
from pyomo.opt import SolverFactory
from classiq import show, synthesize
from classiq import execute
import pandas as pd
from classiq.applications.combinatorial_optimization import get_optimization_solution_from_pyo
from classiq import set_execution_preferences
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences
from classiq import write_qmod
import random as rd

In [139]:
import pickle
import numpy as np

In [140]:
def solve_ksat_class(matrix):
    model = ConcreteModel()
    n, m = len(matrix[0]), len(matrix)
    model.x = Var(range(n), domain=Binary)
    model.obj = Objective(expr=0, sense=minimize)
    model.constraints = ConstraintList()

    for i in range(m):
        clause_expr = 0
        for j in range(n):
            if matrix[i][j] == 1:
                clause_expr += model.x[j]  # Variable itself
            elif matrix[i][j] == -1:
                clause_expr += (1 - model.x[j])  # Negation of the variable
        # Add at least one true literal per clause
        model.constraints.add(clause_expr >= 1)
        solver = SolverFactory('glpk')
        solver.solve(model, tee=False)
    solarray = [int(model.x[i].value) for i in range(n)]
    print(f"Classical solution : {solarray}")
    return [int(model.x[i].value) for i in range(n)]


In [141]:
def verify_solution(matrix, solution):
    num_clauses = len(matrix)
    num_vars = len(matrix[0])
    
    for i in range(num_clauses):
        clause_satisfied = False
        for j in range(num_vars):
            if matrix[i][j] != 0:
                # Determine the truth value of the j-th variable in this clause
                literal_truth = (solution[j] == 1) if matrix[i][j] == 1 else (solution[j] == 0)
                if literal_truth:
                    clause_satisfied = True
                    break  # No need to check more literals in this clause
        if not clause_satisfied:
            return False  # If any clause is not satisfied, return False immediately
    
    return True  # All clauses are satisfied

In [142]:
import random as rd
def generate_ksat_mat(k, m, p=0.5):
       while True: 
        try:
            matrix = np.zeros((m, k), dtype=int)
            unique_rows = set()
            negation_rows = set()
        
            for i in range(m):
                while True:
                    row = np.zeros(k, dtype=int)
                    for j in range(k):
                        if rd.random() < p:  # Probability of including a variable in a clause
                            # Randomly assign either 1 or -1 (true or negated)
                            row[j] = 1 if rd.random() < 0.5 else -1
        
                    row_tuple = tuple(row)
                    negated_row_tuple = tuple(-x for x in row)  # Create a tuple of negated values
        
                    # Check that the row is not all zeros, not already included, and not a negation of an included row
                    if row_tuple not in unique_rows and np.any(row != 0) and negated_row_tuple not in unique_rows:
                        unique_rows.add(row_tuple)
                        negation_rows.add(negated_row_tuple)  # Keep track of negations to avoid adding them later
                        matrix[i, :] = row
                        break
            row_to_append = np.ones(k, dtype=int)
            matrix = np.vstack([matrix, row_to_append])
            solve_ksat_class(matrix.tolist())
            return matrix.tolist()
        except Exception as e:
            print('REEEEEEEEDDOOOOOOOO')

In [143]:
generate_ksat_mat(5,5)

    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 0, 0, 1, 0]


[[0, 0, 1, 0, -1],
 [-1, 0, 0, 1, -1],
 [0, 0, 1, 1, 0],
 [1, 0, 0, 1, 0],
 [-1, 0, -1, 1, -1],
 [1, 1, 1, 1, 1]]

In [265]:
third_case=[]
for k in range(3,16):
    temp = []
    for _ in range(15):
        temp.append(generate_ksat_mat(k=k,m=6))
    third_case.append(temp)

    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 1, 0]
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 0, 1]
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 1, 0]
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 1, 0]
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 1, 1]
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
Class

In [266]:
with open('third_case.pkl', 'wb') as f:
    pickle.dump(third_case, f)

In [99]:
with open('first_case.pkl', 'rb') as f:
    loaded_list = pickle.load(f)

In [101]:
loaded_list[0][4]

[[0, 0, 1], [1, 1, -1], [0, -1, 0], [1, 1, 1]]

In [529]:
import numpy as np

#problem = [[1,0,0,0,0],[1,0,1,0,0],[0,-1,0,0,0],[0,0,0,-1,0],[0,0,1,1,0],[0,0,0,0,-1],[1,0,0,0,1]]
problem = loaded_list[0][12]

In [530]:
problem

[[-1, 1, -1], [1, 1, 0], [0, 0, -1], [1, 1, 1]]

In [531]:
def create_formula(problem):
    string = ''
    for row in problem:
        temp = []
        counter = 1
        for element in row:
            if element == 1:
                temp.append('x' + str(counter))
            elif element == -1:
                temp.append('not x' + str(counter))
            counter += 1

        if len(temp) == 1:
            string += '(' + temp[0] + ')' + 'and'
        elif len(temp) > 1:
            string += '(' + ' or '.join(temp) + ')' + 'and'
    return '(' + string[:-3] + ')'

In [532]:
formula1 = create_formula(problem)

In [538]:
formula1

'((not x1 or x2 or not x3)and(x1 or x2)and(not x3)and(x1 or x2 or x3))'

In [539]:
from classiq import RegisterUserInput, construct_grover_model

register_size = RegisterUserInput(size=1)

qmod = construct_grover_model(
    num_reps=1,
    expression="(" + formula1 + ")",
    definitions=[
        ("x1", register_size),
        ("x2", register_size),
        ("x3", register_size),
    ],
)

In [540]:
from classiq import write_qmod

write_qmod(qmod, "4_sat_grover")

In [551]:
from classiq import QuantumProgram, synthesize, create_model, Preferences, set_preferences

preferences = Preferences(
    backend_service_provider="IBM Quantum", backend_name="ibm_sherbrooke",
    timeout_seconds=900
)
model = set_preferences(qmod, preferences)
qprog = synthesize(model)

In [552]:
circuit = QuantumProgram.from_qprog(qprog)

In [553]:
print(circuit.transpiled_circuit.count_ops)

{'rz': 358, 'sx': 257, 'ecr': 233, 'x': 215}


In [566]:
circuit = circuit.transpiled_circuit


In [ ]:
backend = QasmSimulator()
backend_options = {"method": "statevector"}
 
# Circuit execution
job = execute(circuits, backend, backend_options=backend_options)

In [564]:
circuit.show()

Opening: https://platform.classiq.io/circuit/80d50b65-4987-4b27-b0aa-6ce6588e7ba3?version=0.40.0


In [555]:
from classiq import execute, set_quantum_program_execution_preferences
from classiq.execution import (
    ClassiqBackendPreferences,
    ClassiqSimulatorBackendNames,
    ExecutionPreferences,
)

backend_preferences = ExecutionPreferences(
    backend_preferences=ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR
    )
)

qprog = set_quantum_program_execution_preferences(qprog, backend_preferences)
optimization_result = execute(qprog).result()

In [556]:
qprog.

'{"outputs": {"qasm": "// Generated by Classiq.\\n// Classiq version: 0.40.0\\n// Creation timestamp: 2024-04-21T09:35:13.535839+00:00\\n// Random seed: 3534806905\\n\\nOPENQASM 2.0;\\ninclude \\"qelib1.inc\\";\\ngate c3sx a,b,c,d {\\n  c3sqrtx a,b,c,d;\\n}\\n\\ngate global_phase q0 {\\n  p(2*pi) q0;\\n  rz(-2*pi) q0;\\n}\\n\\ngate grover_search_grover_operator_ugate q0 {\\n  global_phase q0;\\n  u(0,0,0) q0;\\n}\\n\\ngate grover_search_grover_operator_grover_diffuser_uncompute_hadamard_transform_apply_to_all_repeat q0,q1,q2 {\\n  h q0;\\n  h q1;\\n  h q2;\\n}\\n\\ngate grover_search_grover_operator_grover_diffuser_uncompute_hadamard_transform_apply_to_all q0,q1,q2 {\\n  grover_search_grover_operator_grover_diffuser_uncompute_hadamard_transform_apply_to_all_repeat q0,q1,q2;\\n}\\n\\ngate grover_search_grover_operator_grover_diffuser_uncompute_hadamard_transform q0,q1,q2 {\\n  grover_search_grover_operator_grover_diffuser_uncompute_hadamard_transform_apply_to_all q0,q1,q2;\\n}\\n\\ngate

In [546]:
res = optimization_result[0].value

In [548]:
result = res.counts_of_multiple_outputs(("x1", "x2", "x3"))

In [549]:
result

{('0', '1', '1'): 66,
 ('0', '0', '1'): 66,
 ('1', '0', '0'): 554,
 ('1', '0', '1'): 63,
 ('0', '0', '0'): 56,
 ('0', '1', '0'): 612,
 ('1', '1', '1'): 59,
 ('1', '1', '0'): 572}

In [477]:
max_key = max(result, key=result.get)
maxkey = list([int(x) for x in max_key])
maxkey

[0, 1, 1, 0, 1]

In [485]:
verify_solution(loaded_list[2][8],[1,1,1,1,0])

False

In [550]:
loaded_list[0][12]


[[-1, 1, -1], [1, 1, 0], [0, 0, -1], [1, 1, 1]]